In [ ]:
%load_ext autoreload
%autoreload 2

import os
import yaml
import time
import torch
import random
import numpy as np
from tqdm import tqdm

from torch.utils import data

from tensorboardX import SummaryWriter

import data_loader
import models
from optimizer import get_optimizer
from loss.cross_entropy_2d import CrossEntropy2d
from utils.metrics import MetricsComp, AverageComp

def train(config, writer=None):
    # setup seeds
    torch.manual_seed(config["seed"])
    torch.cuda.manual_seed(config["seed"])
    np.random.seed(config["seed"])
    torch.cuda.empty_cache()
    
    # setup device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # setup augmentations
    ## ah todo
    
    # setup dataloader
    sim_data = data_loader.get_loader("simulated")()
    train_data = data.DataLoader(sim_data, batch_size=config["batch_sz"],\
                                 num_workers=config["num_workers"], shuffle=True)
    val_data = data.DataLoader(data_loader.get_loader("simulated")(n_imgs=10), batch_size=config["batch_sz"],\
                               num_workers=config["num_workers"])  
    n_classes = sim_data.n_classes
    
    # setup metrics
    metrics_comp = MetricsComp(n_classes)
    val_loss_avg_comp = AverageComp()
    
    # setup model
    model = models.get_model("fcn32s", n_classes).to(device)
    #model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    
    # setup optimizer
    optimizer = get_optimizer("adam")(model.parameters())
    
    print(f"Optimizer: {optimizer}")
    
    # setup learning rate scheduler (optim.lr_scheduler)
    ## ah todo
    weight = torch.tensor([1.0] +  9*[10.0]).to(device) # weight the background 10x less
    loss_fn = CrossEntropy2d(weight=weight)
    
    writer.add_scalar("batch_size", config["batch_sz"])

    i = 0
    # Load a saved checkpoint
    if config.get("resume_ckpoint") is not None:
        if os.path.isfile(config["resume_ckpoint"]):
            print(f"Loading checkpoint: {config['resume_ckpoint']}")
            ckpoint = torch.load(config["resume_ckpoint"])
            model.load_state_dict(ckpoint["model_state"])
            optimizer.load_state_dict(ckpoint["optimizer_state"])
            i = ckpoint["epoch"]
            print(f"Saved epoch: {i}, loss: {ckpoint['epoch_loss']}, time: {ckpoint['epoch_time']}")
        else:
            raise FileNotFoundError("Unable to load saved checkpoint!, Quitting")
            
    while i < config["epochs"]:
        i += 1
        epoch_loss = 0
        epoch_time = 0
        print(f"Epoch {i}")
        # Prepare for training
        model.train()
        optimizer.zero_grad()
        acc_gradients_batch = 0
        for b_i, (images, labels) in tqdm(enumerate(train_data)):
            start_ts = time.time()
            images = images.to(device)
            labels = labels.to(device)
            
            out = model(images) # [batch_sz, n_classes, H=512, W=512]
            acc_gradients_batch += out.shape[0]

            loss = loss_fn(out, labels)    
            loss.backward()
            
            # See if it is time to take a gradient step.
            if acc_gradients_batch >= 100: # gradient step every 100 samples
              optimizer.step()
              optimizer.zero_grad()
              acc_gradients_batch = 0
              
            epoch_loss += float(loss.item())
            epoch_time += time.time() - start_ts
            writer.add_scalar(f"{i}_batch_loss", loss.item(), b_i)
        
        print(f"Epoch Loss: {epoch_loss}")
          
        # Run through validation
        model.eval()
        with torch.no_grad():
            ctr = 0
            for _, (images_val, labels_val) in tqdm(enumerate(val_data)):
                if ctr > 10:
                    break
                ctr += 1
                images_val = images_val.to(device)
                labels_val = labels_val.to(device)               
                out = model(images_val) # [batch_sz, n_classes, H=512, W=512]
                val_loss = loss_fn(out, labels_val)
                val_loss_avg_comp.update(val_loss.item())
                
                _, pred = out.max(1)

                pred_uv, pred_uc = np.unique(pred.cpu().numpy(), return_counts=True)
                ps = [f"{p_uv} ({p_uc})" for p_uv, p_uc in zip(pred_uv, pred_uc)]
                print(f"pred: {ps}")
                
                lbl_uv, lbl_uc = np.unique(labels_val.cpu().numpy(), return_counts=True)
                lbls = [f"{l_uv} ({l_uc})" for l_uv, l_uc in zip(lbl_uv, lbl_uc)]
                print(f"labels_val: {lbls}")

                metrics_comp.update(label_trues=labels_val.cpu().numpy(), label_preds=pred.cpu().numpy())
        # Add validation results to tensorboard writer
        writer.add_scalar("val_loss", val_loss_avg_comp.avg, i)
        overall_scores, class_iou = metrics_comp.get_results()
        print(f"Scores after epoch {i}: {overall_scores}")
        for k, v in overall_scores.items():
            writer.add_scalar(f"val_metrics/{k}", v, i)
        for k, v in class_iou.items():
            writer.add_scalar(f"val_metrics/cls_iou_{k}", v, i)
        
        # Save the model checkpoint
        ckpoint = {"epoch":i,
                   "model_state": model.state_dict(),
                   "optimizer_state": optimizer.state_dict(),
                   "epoch_loss": epoch_loss,
                   "epoch_time": epoch_time,
                  }
        ckpoint_name = f"checkpoints/{config['exp_name']}_{i%3}.pkl"
        torch.save(ckpoint, ckpoint_name)
    
            
config = {"exp_name": "fcn32s", "batch_sz": 8, "epochs": 5000,\
          "seed": 3642, "num_workers": 1, "resume_ckpoint": "checkpoints/fcn32s_0.pkl"}
writer = SummaryWriter(log_dir="checkpoints/logs")

train(config, writer)

Generating 500 images
Generating 10 images
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
********  weight: tensor([ 1., 10., 10., 10., 10., 10., 10., 10., 10., 10.], device='cuda:0')
********  weight: tensor([ 1., 10., 10., 10., 10., 10., 10., 10., 10., 10.], device='cuda:0')
Loading checkpoint: checkpoints/fcn32s_0.pkl
Saved epoch: 39, loss: 15.323740854859352, time: 92.98452639579773
Epoch 40


63it [01:33,  1.49s/it]

Epoch Loss: 14.441147327423096



1it [00:00,  1.19it/s]

pred: ['0 (1760038)', '2 (203314)', '3 (133800)']
labels_val: ['0 (1903905)', '1 (1639)', '2 (132736)', '3 (58872)']


...

pred: ['0 (1788359)', '2 (143761)', '3 (165032)']
labels_val: ['0 (1931133)', '1 (1894)', '2 (95001)', '3 (69124)']


2it [00:01,  1.82it/s]

pred: ['0 (445387)', '2 (46381)', '3 (32520)']
labels_val: ['0 (482625)', '1 (249)', '2 (31242)', '3 (10172)']
Scores after epoch 47: {'Overall Acc: ': 0.9297134399414062, 'FreqW Acc : ': 16378950.493991993, 'Mean IoU : ': 1949750.4}


Epoch 48


36it [00:54,  1.48s/it]

In [ ]:
!pwd

In [ ]:
!ls -alh checkpoints/logs